In [1]:
#!pip install tqdm
import torch
import torchvision # torch package for vision related things
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
import torchvision.datasets as datasets  # Standard datasets
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
from torch.utils.data import DataLoader  # Gives easier dataset managment by creating mini batches etc.
from tqdm import tqdm  # For nice progress bar!

In [9]:
#define model_parameters
input_size=784
num_classes=10
learning_rate=0.005
batch_size=64
num_epochs=3
#setup device
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
#Load The Data
train_data=datasets.MNIST(root='dataset/',train=True,transform=transforms.ToTensor(),download=True)
test_data=datasets.MNIST(root='dataset/',train=False,transform=transforms.ToTensor(),download=True)
train_loader=DataLoader(dataset=train_data,batch_size=batch_size,shuffle=True)
test_loader=DataLoader(dataset=test_data,batch_size=batch_size,shuffle=True)

In [7]:
class CNN(nn.Module):
    def __init__(self,n_channels=1,num_classes=10):
        super(CNN,self).__init__()
        self.conv1=nn.Conv2d(in_channels=1,out_channels=8,kernel_size=(3,3),stride=(1,1),padding=(1,1))
        self.pool=nn.MaxPool2d(kernel_size=(2,2),stride=(2,2))
        self.conv2=nn.Conv2d(in_channels=8,out_channels=16,kernel_size=(3,3),stride=(1,1),padding=(1,1))
        self.fc1=nn.Linear(16*7*7,num_classes)
    def forward(self,x):
        x=F.relu(self.conv1(x))
        x=self.pool(x)
        x=F.relu(self.conv2(x))
        x=self.pool(x)
        x=x.reshape(x.shape[0],-1)
        x=self.fc1(x)
        return x
try:
    model=CNN()
    x=torch.randn(64,1,28,28)
    model(x)
    print("Sanity Check: Passed!")
except:
    print("Sanity Check Failed")

Sanity Check: Passed!


In [10]:
model=CNN().to(device)
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=learning_rate)

In [13]:
for epoch in range(num_epochs):
    for batch_idx,(data,targets) in enumerate(tqdm(train_loader)):
        #get data to cuda
        data=data.to(device=device)
        targets=targets.to(device=device)
        #forward
        scores=model(data)
        loss=criterion(scores,targets)
        #backward
        optimizer.zero_grad()
        loss.backward()
        #gradient Descent
        optimizer.step()
    if(epoch==num_epochs-1):
        print("Training Finished")

100%|███████████████████████████████████████████████████████████████████████████████| 938/938 [00:06<00:00, 155.27it/s]

Training Finished


In [16]:
def check_acc(loader,model):
    #initialize the accuracy testing variables
    correct=0
    samples=0
    #set the model to evaluation stage
    model.eval() 
    with torch.no_grad():
        for x,y in loader:
            x=x.to(device=device)
            y=y.to(device=device)
            
            scores=model(x)
            _,predictions=scores.max(1)
            correct += (predictions == y).sum()
            samples += predictions.size(0)
    model.train()
    return correct/samples

In [21]:
print("Training Accuracy:{0}".format(check_acc(train_loader,model)))
print("Testing Accuracy:{0}".format(check_acc(test_loader,model)))

Training Accuracy:0.9869666695594788
Testing Accuracy:0.9818999767303467
